In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Set Project Root

In [2]:
import os

PROJECT_ROOT = "/content/drive/MyDrive/GalaxEye_ship_detection/galaxeye"
os.chdir(PROJECT_ROOT)

print("Working directory:", os.getcwd())
print("Contents:", os.listdir("."))

Working directory: /content/drive/MyDrive/GalaxEye_ship_detection/galaxeye
Contents: ['scripts', 'requirements.txt', 'yolov8s.pt', 'results', 'runs', 'data', 'yolo26n.pt']


### Install GEE Python API

In [2]:
!pip install earthengine-api geemap rasterio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.3/476.3 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.4/113.4 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 91.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.8/231.8 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 101.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1

### Authenticate & Initialize Earth Engine

In [1]:
import ee

ee.Authenticate()
ee.Initialize(project='ee-deepeshy')

print("✅ Earth Engine initialized successfully!")

✅ Earth Engine initialized successfully!


## Define Area of Interest (AOI)

### **Panama AOI**

In [17]:
import ee
import geemap

# Panama Canal – Pacific entrance (open sea, high ship density)
# Center point
center_lat = 8.813066
center_lon = -79.489989

# Small square size (~2km x 2km)
d = 0.01  # ~0.01 degrees ≈ 1.1 km (so total box ≈ 2.2 km)

# AOI rectangle: [lon_min, lat_min, lon_max, lat_max]
aoi_panama = ee.Geometry.Rectangle([
    center_lon - d, center_lat - d,
    center_lon + d, center_lat + d
])

aoi_panama

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "GeometryConstructors.Polygon",
    "arguments": {
      "coordinates": {
        "constantValue": [
          [
            [
              -79.499989,
              8.823065999999999
            ],
            [
              -79.499989,
              8.803066
            ],
            [
              -79.47998899999999,
              8.803066
            ],
            [
              -79.47998899999999,
              8.823065999999999
            ]
          ]
        ]
      },
      "evenOdd": {
        "constantValue": true
      }
    }
  }
})

### Load Sentinel-1 (Filtered Correctly)

In [18]:
sentinel1_panama = (
    ee.ImageCollection("COPERNICUS/S1_GRD")
    .filterBounds(aoi_panama)
    .filterDate("2023-01-01", "2023-12-31")
    .filter(ee.Filter.eq("instrumentMode", "IW"))
    .filter(ee.Filter.listContains("transmitterReceiverPolarisation", "VV"))
    .filter(ee.Filter.eq("orbitProperties_pass", "DESCENDING"))
    .select("VV")
)

print("Sentinel-1 images:", sentinel1_panama.size().getInfo())

Sentinel-1 images: 29


### Convert to dB Scale

In [19]:
def to_db(image):
    return ee.Image(10).multiply(image.log10()) \
        .copyProperties(image, ["system:time_start"])

sentinel1_panama_db = sentinel1_panama.map(to_db)

### Temporal Mean (Speckle Reduction)

In [20]:
sentinel1_panama_mean = (
    sentinel1_panama_db
    .mean()
    .clip(aoi_panama)
)

sentinel1_panama_mean

### Visualization of AOI on the Map

In [21]:
Map = geemap.Map(center=[center_lat, center_lon], zoom=12)

Map.addLayer(
    sentinel1_panama_mean,
    {"min": -25, "max": 0},
    "Sentinel-1 VV (dB) – Panama"
)

# Showing AOI boundary
Map.addLayer(aoi_panama, {"color": "blue"}, "Small AOI Square")

Map

Map(center=[8.813066, -79.489989], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sear…

### Export to Google Drive (FINAL)

In [22]:
task = ee.batch.Export.image.toDrive(
    image=sentinel1_panama_mean,
    description="Sentinel1_VV_Panama",
    folder="sentinel1_exports",
    fileNamePrefix="sentinel1_vv_panama",
    region=aoi_panama,
    scale=10,
    crs="EPSG:4326",
    maxPixels=1e13
)

task.start()
print("Panama Sentinel-1 export started…")

Panama Sentinel-1 export started…


### Monitor Export

In [23]:
task.status()

{'state': 'COMPLETED',
 'description': 'Sentinel1_VV_Panama',
 'priority': 100,
 'creation_timestamp_ms': 1769421661050,
 'update_timestamp_ms': 1769421789142,
 'start_timestamp_ms': 1769421673187,
 'task_type': 'EXPORT_IMAGE',
 'destination_uris': ['https://drive.google.com/#folders/1F2O_3KWzp0WWduF-dE3adTOUos4e1pNM'],
 'attempt': 1,
 'batch_eecu_usage_seconds': 6.171223163604736,
 'id': 'WHL5JL3OYBF2O4BOHXSYGWRK',
 'name': 'projects/ee-deepeshy/operations/WHL5JL3OYBF2O4BOHXSYGWRK'}